## Machine-learning on Options

In [ ]:
#!pip install import_ipynb --quiet

In [ ]:
#!git clone https://github.com/gmshroff/mlfin2022.git

In [ ]:
# !rm models.ipynb
# from google.colab import files
# files.upload()
# #upload your own models.ipynb

In [ ]:
import pandas as pd
import import_ipynb
import utils
import models
import options_data
import optutils

In [ ]:
from IPython import display
import torch
from sklearn.manifold import TSNE
from matplotlib import pyplot as plt
from IPython import display
import numpy as np
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve
from sklearn.metrics import auc
import pickle

## Loss Function

In [ ]:
lossfn=torch.nn.NLLLoss()

In [ ]:
nll=torch.nn.NLLLoss(weight=torch.tensor([0.1,0.9]))
softAUC=optutils.SoftAUC(scale=10.)
WLoss=optutils.WeightedLoss(nll,softAUC.loss,a=1,b=100)
lossfn=WLoss.loss

# Get Options Data

In [ ]:
OPTYPE='CE'

Pre-loaded meta-learning datasets:

In [ ]:
with open('/Users/a112956/DataLocal/meta-options/opDS'+OPTYPE+'_train2017_test2019.pickle','rb') as fh:
    opDS=pickle.load(fh)

In [ ]:
len(opDS.meta_train_tasks), len(opDS.meta_test_tasks),

## Standard MLP 

Define MLP model and create normal train-test data

In [ ]:
net0 = models.MLP(dims=[148,256,32,2])

In [ ]:
train_loader,trainDS,testDS=opDS.normal_train_test(minDate='01-Jan-2017',
                                                  maxTrainDate='31-Dec-2018',
                                                   minTestDate='01-Mar-2019',
                                                  maxTestDate='31-Dec-2019')

In [ ]:
print(trainDS.samples.shape,testDS.samples.shape[0])

Load saved MLP model

In [ ]:
net0.load_state_dict(torch.load('./savedmodels/MLP2017'+OPTYPE+'.pth'))

Train MLP model

In [ ]:
net0,losses,accs=models.Train(net0,train_loader,epochs=20,verbose=True,Loss=lossfn)

In [ ]:
torch.save(net0.state_dict(),'./savedmodels/MLP2017'+OPTYPE+'.pth')

In [ ]:
models.accuracy(net0,testDS.samples,testDS.labels,verbose=True)

In [ ]:
labels=testDS.labels.detach().numpy()
sm=torch.nn.Softmax(dim=1)
probs = sm(net0(testDS.samples)).detach().numpy()
pr,re,th=precision_recall_curve(labels,probs[:,1])
ix=np.argsort(th)
plt.plot(th[ix],pr[ix],color='red')
plt.plot(th[ix],re[ix],color='blue')
plt.show()

In [ ]:
labels=testDS.labels.detach().numpy()
sm=torch.nn.Softmax(dim=1)
probs = sm(net0(testDS.samples)).detach().numpy()
pr,re,th=precision_recall_curve(labels,probs[:,1])
ix=np.argsort(th)
plt.plot(th[ix],pr[ix],color='red')
plt.plot(th[ix],re[ix],color='blue')
plt.show()

In [ ]:
th[ix]